In [1]:
# Imports
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Hyperparameters
input_size = 28
hidden_size = 256
num_layers = 2
num_classes = 10
sequence_length = 28
learning_rate = 0.005
batch_size = 64
num_epochs = 2

In [16]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        print("x shape : ",x.shape)

        # Forward propagate LSTM
        out, _ = self.rnn(x, h0)

        print("out shape : ",out.shape)

        out = out.reshape(out.shape[0], -1)

        print("out shape2:",out.shape)
        # Decode the hidden state of the last time step
        out = self.fc(out)

        print("out shape3 : ",out.shape)

        return out


In [4]:
# Recurrent neural network with LSTM (many-to-one)
class RNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        

        # Forward propagate LSTM
        out, _ = self.lstm(
            x, (h0, c0)
        )  # out: tensor of shape (batch_size, seq_length, hidden_size)
        out = out.reshape(out.shape[0], -1)

        

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out

In [12]:
# Load Data
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)

test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [18]:
# Initialize network
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
for im,l in train_loader:
  print(im.shape)
  break

torch.Size([64, 1, 28, 28])


In [19]:
# Train Network
general_loss = []
for epoch in range(num_epochs):
    epoch_loss = []
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        epoch_loss.append(loss.item())
        
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()


    mean_loss = sum(epoch_loss)/len(epoch_loss)
    general_loss.append(mean_loss)
    print(f"epoch loss {mean_loss} in {epoch}")    

x shape :  torch.Size([64, 28, 28])
out shape :  torch.Size([64, 28, 256])
out shape2: torch.Size([64, 7168])
out shape3 :  torch.Size([64, 10])
x shape :  torch.Size([64, 28, 28])
out shape :  torch.Size([64, 28, 256])
out shape2: torch.Size([64, 7168])
out shape3 :  torch.Size([64, 10])
x shape :  torch.Size([64, 28, 28])
out shape :  torch.Size([64, 28, 256])
out shape2: torch.Size([64, 7168])
out shape3 :  torch.Size([64, 10])
x shape :  torch.Size([64, 28, 28])
out shape :  torch.Size([64, 28, 256])
out shape2: torch.Size([64, 7168])
out shape3 :  torch.Size([64, 10])
x shape :  torch.Size([64, 28, 28])
out shape :  torch.Size([64, 28, 256])
out shape2: torch.Size([64, 7168])
out shape3 :  torch.Size([64, 10])
x shape :  torch.Size([64, 28, 28])
out shape :  torch.Size([64, 28, 256])
out shape2: torch.Size([64, 7168])
out shape3 :  torch.Size([64, 10])
x shape :  torch.Size([64, 28, 28])
out shape :  torch.Size([64, 28, 256])
out shape2: torch.Size([64, 7168])
out shape3 :  torch.

KeyboardInterrupt: ignored